In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

In [2]:
# Load the dataset
file_path = '/Users/mahinbindra/Downloads/sp500_data.csv'
sp500_data = pd.read_csv(file_path)

In [3]:
# Preprocessing to identify sectors and locations
sectors = sp500_data['GICS Sector'].unique()
companies = sp500_data['Ticker symbol'].tolist()
returns = sp500_data.set_index('Ticker symbol')['PercentReturn'].to_dict()
sector_companies = {sector: sp500_data['Ticker symbol'][sp500_data['GICS Sector'] == sector].tolist() for sector in sectors}
ny_companies = sp500_data['Ticker symbol'][sp500_data['Location of Headquarters'].str.contains('New York, New York')].tolist()

In [4]:
# Initialize the model
m = gp.Model("investment_portfolio")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-15


In [5]:
# Decision Variables
invest = m.addVars(companies, name="invest")

In [6]:
# Objective Function
m.setObjective(gp.quicksum(invest[c] * returns[c] for c in companies), GRB.MAXIMIZE)

In [7]:
# Constraints
# Total investment of $10 million
m.addConstr(gp.quicksum(invest[c] for c in companies) == 10e6, "total_investment")

# At most $600,000 can be invested in any individual stock
m.addConstrs((invest[c] <= 600000 for c in companies), "max_invest_individual")

# No more than $500,000 can be invested in the Telecommunications sector
telecom_sector = "Telecommunications Services"
m.addConstr(gp.quicksum(invest[c] for c in sector_companies[telecom_sector]) <= 500000, "max_invest_telecom")

# At least 75% in IT compared to Telecommunications
it_sector = "Information Technology"
m.addConstr(gp.quicksum(invest[c] for c in sector_companies[it_sector]) >= 0.75 * gp.quicksum(invest[c] for c in sector_companies[telecom_sector]), "min_invest_it")

# Difference between Consumer Discretionary and Consumer Staples
consumer_discretionary = "Consumer Discretionary"
consumer_staples = "Consumer Staples"
m.addConstr(gp.quicksum(invest[c] for c in sector_companies[consumer_discretionary]) - gp.quicksum(invest[c] for c in sector_companies[consumer_staples]) <= 200000, "consumer_difference_pos")
m.addConstr(gp.quicksum(invest[c] for c in sector_companies[consumer_staples]) - gp.quicksum(invest[c] for c in sector_companies[consumer_discretionary]) <= 200000, "consumer_difference_neg")

# At least $1 million must be invested in the Energy sector
energy_sector = "Energy"
m.addConstr(gp.quicksum(invest[c] for c in sector_companies[energy_sector]) >= 1e6, "min_invest_energy")

# At least $300,000 must be invested in companies headquartered in New York, New York
m.addConstr(gp.quicksum(invest[c] for c in ny_companies) >= 300000, "min_invest_ny")

<gurobi.Constr *Awaiting Model Update*>

In [8]:
# Solve the model
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 74 rows, 67 columns and 196 nonzeros
Model fingerprint: 0xae85b2df
Coefficient statistics:
  Matrix range     [8e-01, 1e+00]
  Objective range  [1e-01, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+05, 1e+07]
Presolve removed 68 rows and 16 columns
Presolve time: 0.00s
Presolved: 6 rows, 52 columns, 92 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4560000e+08   3.474937e+06   0.000000e+00      0s
       6    5.1346000e+07   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.01 seconds (0.00 work units)
Optimal objective  5.134600000e+07


In [9]:
# Display the solution
if m.status == GRB.OPTIMAL:
    solution = m.getAttr('x', invest)
    for c in companies:
        if solution[c] > 0:
            print(f"Invest ${solution[c]:,.2f} in {c} ({sp500_data.loc[sp500_data['Ticker symbol'] == c, 'Company'].iloc[0]})")

Invest $600,000.00 in AEE (Ameren Corp)
Invest $600,000.00 in AVP (Avon Products)
Invest $400,000.00 in CVC (Cablevision Systems Corp.)
Invest $600,000.00 in COP (ConocoPhillips)
Invest $600,000.00 in RRD (Donnelley (R.R.) & Sons)
Invest $600,000.00 in DUK (Duke Energy)
Invest $400,000.00 in XOM (Exxon Mobil Corp.)
Invest $500,000.00 in FTR (Frontier Communications)
Invest $600,000.00 in HCP (HCP Inc.)
Invest $600,000.00 in KIM (Kimco Realty)
Invest $600,000.00 in MRK (Merck & Co.)
Invest $375,000.00 in MCHP (Microchip Technology)
Invest $600,000.00 in PBCT (People's United Bank)
Invest $600,000.00 in POM (Pepco Holdings Inc.)
Invest $600,000.00 in PCG (PG&E Corp.)
Invest $525,000.00 in PNW (Pinnacle West Capital)
Invest $600,000.00 in PCL (Plum Creek Timber Co.)
Invest $600,000.00 in TE (TECO Energy)


In [10]:
# Count the number of companies headquartered in "New York, New York"
ny_ny_companies_count = sp500_data['Location of Headquarters'].str.contains('New York, New York').sum()
print(f"Number of companies headquartered in 'New York, New York': {ny_ny_companies_count}")

Number of companies headquartered in 'New York, New York': 6


In [11]:
# Filter companies headquartered in "New York, New York"
ny_ny_companies = sp500_data[sp500_data['Location of Headquarters'].str.contains('New York, New York')]['Ticker symbol']
# Sum the investment in these companies
total_invested_in_ny_ny = sum(solution[c] for c in ny_ny_companies if c in solution)
print(f"Total amount invested in companies headquartered in 'New York, New York': ${total_invested_in_ny_ny:,.2f}")

Total amount invested in companies headquartered in 'New York, New York': $600,000.00


In [12]:
# Accessing the shadow price (dual value) of the Energy sector investment constraint
energy_sector_shadow_price = m.getConstrByName("min_invest_energy").Pi
print(f"Shadow price for the Energy sector investment constraint: {energy_sector_shadow_price}%")

# Decision advice based on the shadow price
if energy_sector_shadow_price < 0:
    print("Reducing the minimum investment in the Energy sector will increase the portfolio's expected return.")
else:
    print("Increasing the minimum investment in the Energy sector could potentially reduce the portfolio's expected return.")

Shadow price for the Energy sector investment constraint: -2.14%
Reducing the minimum investment in the Energy sector will increase the portfolio's expected return.


In [13]:
# Find the original return rate for Coca-Cola Enterprises
# Assuming 'Company' column contains the company names and 'PercentReturn' the return rates
coca_cola_return = sp500_data.loc[sp500_data['Company'] == 'Coca-Cola Enterprises', 'PercentReturn'].iloc[0]

# New information
new_return = 3.00  # New return rate for Coca-Cola Enterprises is 3.00%
allowable_increase = 2.4  # 2.4%

# Convert the allowable increase to a proportion for comparison
allowable_increase_proportion = allowable_increase 

# Original return rate for comparison (assumed to be in percent in the dataset)
original_return = coca_cola_return  # Convert to a proportion if needed

# Check if the new return is within the allowable change range
if original_return <= new_return <= original_return + allowable_increase_proportion:
    print("No need to change the portfolio based on the new return information.")
else:
    print("Consider re-evaluating the portfolio allocation.")

No need to change the portfolio based on the new return information.


In [14]:
# Assuming 'm' is your Gurobi model object and it has been optimized
telecom_constraint = m.getConstrByName("max_invest_telecom")
telecom_sector_shadow_price = telecom_constraint.Pi

print(f"Shadow price for the Telecommunications sector constraint: {telecom_sector_shadow_price}%")

proposed_increase = 125000  # From $500,000 to $625,000 is a $125,000 increase
allowable_increase = 100000  # Given as part of the problem statement

# Evaluate the decision
if proposed_increase <= allowable_increase:
    print("The proposed increase is within the allowable range. It could be a good decision based on the positive shadow price.")
else:
    print("The proposed increase exceeds the allowable increase. This requires reevaluation of the portfolio.")

Shadow price for the Telecommunications sector constraint: 9.78%
The proposed increase exceeds the allowable increase. This requires reevaluation of the portfolio.


In [15]:
# Retrieve the shadow price for the total investment constraint
total_investment_constraint = m.getConstrByName("total_investment")
shadow_price_total_investment = total_investment_constraint.Pi
print(f"Shadow price for the total investment constraint: {shadow_price_total_investment}")

# Given borrowing terms
loan_amount = 50000  # Maximum loan amount you can borrow
interest_rate = 1.5  # 1.5% interest rate for borrowing
allowable_increase = 75000  # Allowable increase in the total investment from the sensitivity analysis

# Calculate the cost of borrowing
total_repayment = loan_amount * (1 + interest_rate)

# Calculate the potential return on the borrowed amount using the shadow price
# Convert the shadow price to a percentage form if it's not already
shadow_price_percentage = shadow_price_total_investment
potential_return = loan_amount * shadow_price_percentage

# Determine if borrowing is beneficial
if potential_return > total_repayment:
    # Determine the beneficial amount to borrow, considering the allowable increase
    max_borrowing = min(allowable_increase, loan_amount)
    print(f"It's beneficial to borrow up to ${max_borrowing:.2f} because the potential return ({potential_return:.2f}) exceeds the cost of borrowing ({total_repayment:.2f}).")
    amount_to_borrow = max_borrowing
else:
    print("Borrowing is not beneficial because the potential return does not exceed the cost of borrowing.")
    amount_to_borrow = 0

# Output the decision
print(f"Amount willing to borrow based on the analysis: ${amount_to_borrow:.2f}")

Shadow price for the total investment constraint: 4.36
It's beneficial to borrow up to $50000.00 because the potential return (218000.00) exceeds the cost of borrowing (125000.00).
Amount willing to borrow based on the analysis: $50000.00
